In [14]:
import numpy as np
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.layers import Embedding, LSTM, Dense, Bidirectional

import matplotlib.pyplot as plt

Transfer learing with MobileNet




In [15]:
model = tf.keras.applications.MobileNet(
    input_shape=None,
    alpha=1.0,
    depth_multiplier=1,
    dropout=0.001,
    include_top=True,
    weights="imagenet",
    input_tensor=None,
    pooling=None,
    classes=1000,
    classifier_activation="softmax",
)



Blokowania (zamarzanie warstw) 



Warstwy i modele mają trzy atrybuty wagi:

weights - lista wszystkich zmiennych wag warstwy

trainable_weights - lista wag, które mają być aktualizowane

non_trainable_weights - lista nieternowana

In [22]:

layer = keras.layers.Dense(3)
layer.build((None, 4))  

print("na przykład warstwa Dense ma 2 wagi: weights:", len(layer.weights))
print("te które można trenować: trainable_weights:", len(layer.trainable_weights))
print("ale także takie, których trenowanie jest niemozliwe: non_trainable_weights:", len(layer.non_trainable_weights))


na przykład warstwa Dense ma 2 wagi: weights: 2
te które można trenować: trainable_weights: 2
ale także takie, których trenowanie jest niemozliwe: non_trainable_weights: 0


Warstwa batch noemalzation ma 4 wagi: 2 z mozliwością trenowania i 2 nietrenowalne:

In [23]:
layer = keras.layers.BatchNormalization()
layer.build((None, 4))  

print("weights:", len(layer.weights))
print("trainable_weights:", len(layer.trainable_weights))
print("non_trainable_weights:", len(layer.non_trainable_weights))

weights: 4
trainable_weights: 2
non_trainable_weights: 2


Jesli wagi są nietrenowalne, ich wartość nie jest aktualizowana podczas trenowania modelu:

In [34]:
layer1 = keras.layers.Dense(3, activation="relu")
layer2 = keras.layers.Dense(3, activation="sigmoid")
model = keras.Sequential([keras.Input(shape=(3,)), layer1, layer2])

# zamaranie wag warstwy 1 trenowawlnej
layer1.trainable = False

# tworzenie kopi warstwy 1 (aby móc trenować na kopi wag)
initial_layer1_weights_values = layer1.get_weights()


model.compile(optimizer="adam", loss="mse")
model.fit(np.random.random((2, 3)), np.random.random((2, 3)))

final_layer1_weights_values = layer1.get_weights()
np.testing.assert_allclose(
    initial_layer1_weights_values[0], final_layer1_weights_values[0]
)
np.testing.assert_allclose(
    initial_layer1_weights_values[1], final_layer1_weights_values[1]
)


print("trainable_weights:", len(layer1.trainable_weights)) 
print("weights:", len(layer1.weights))
print("non_trainable_weights:", len(layer1.non_trainable_weights))


1/1 [==============================] - 0s 421ms/step - loss: 0.0639
trainable_weights: 0
weights: 2
non_trainable_weights: 2


Musimy trainable ustawić na FALSE

Przykład z modelem Sequential

In [35]:
inner_model = keras.Sequential(
    [
        keras.Input(shape=(3,)),
        keras.layers.Dense(3, activation="relu"),
        keras.layers.Dense(3, activation="relu"),
    ]
)

model = keras.Sequential(
    [keras.Input(shape=(3,)), inner_model, keras.layers.Dense(3, activation="sigmoid"),]
)

#Model zostaje zamrożony
model.trainable = False  

assert inner_model.trainable == False 
assert inner_model.layers[0].trainable == False  

Można zrobić tak z każdym modelem dostępnym na keras API

In [36]:
import tensorflow_datasets as tfds

tfds.disable_progress_bar()

train_ds, validation_ds, test_ds = tfds.load(
    "cats_vs_dogs",
    # Reserve 10% for validation and 10% for test
    split=["train[:40%]", "train[40%:50%]", "train[50%:60%]"],
    as_supervised=True,  # Include labels
)

Dataset cats_vs_dogs downloaded and prepared to C:\Users\mhalama\tensorflow_datasets\cats_vs_dogs\4.0.0. Subsequent calls will reuse this data.


In [38]:
base_model = tf.keras.applications.MobileNet(
    input_shape=None,
    alpha=1.0,
    depth_multiplier=1,
    dropout=0.001,
    include_top=True,
    weights="imagenet",
    input_tensor=None,
    pooling=None,
    classes=1000,
    classifier_activation="softmax"
)

base_model.trainable = False

In [40]:
inputs = keras.Input(shape=(224, 224, 3))
x = base_model(inputs, training=False)
x = keras.layers.GlobalAveragePooling2D()(x)
outputs = keras.layers.Dense(1)(x)
model = keras.Model(inputs, outputs)

ValueError: Input 0 of layer "global_average_pooling2d_7" is incompatible with the layer: expected ndim=4, found ndim=2. Full shape received: (None, 1000)